In [13]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch import optim
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
import copy
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.models import resnet50, resnet34, resnet18, wide_resnet50_2, ResNet50_Weights, alexnet
import gc
import os
import pandas as pd
from torchvision.io import read_image
from flax.training import checkpoints
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor


!pip install wandb
!pip install kornia
!pip install optuna
import optuna
import kornia
import wandb

In [14]:
from networks import ConvNet, AlexNet
from distill import ParamDiffAug
from utils import evaluate_synset, get_network
import argparse
#labels_train = torch.load('/datasets/mtt-cifar10-50-ipc/cifar10_50ipc_labels.pt')
#images_train = torch.load('/datasets/mtt-cifar10-50-ipc/cifar10_50ipc_images.pt')
labels_train = torch.load('./data/cifar10_10ipc_labels.pt')
images_train = torch.load('./data/cifar10_10ipc_images.pt')

In [15]:
batch_size = 256
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                                    train = True,
                                                    transform = transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]),]),
                                                    download=True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)


test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                                    train = False,
                                                    transform = transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]),]),
                                                    download=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)

transform_train = transforms.Compose([transforms.Resize((32,32)),  #resises the image so it can be perfect for our model.
                                      transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
                                      transforms.RandomRotation(10),     #Rotates the image to a specified angel
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
                                      transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]), #Normalize all the images
                               ])


augmented_train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                                    train = True,
                                                    transform = transform_train,
                                                    download=True)

augmented_train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Distilled Training Params: .085 lr and 1300 epochs

In [5]:
#Get pretrained model to determine what is easy/hard
pretrained_model = AlexNet(3,10).to('cuda')
train(pretrained_model, test_loader, 60)

In [15]:
model = AlexNet(3,10).to('cuda')
args = argparse.Namespace(lr_net=str(.007), device='cuda', epoch_eval_train=str(500),batch_train=100,dataset='cifar10',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True))                        
for i in range(20):
    model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)

100%|██████████| 501/501 [00:08<00:00, 58.85it/s]


[2023-05-30 02:14:34] Evaluate_01: epoch = 0500 train time = 8 s train loss = 2.299133 train acc = 0.0700, test acc = 0.1009


100%|██████████| 501/501 [00:08<00:00, 59.84it/s]


[2023-05-30 02:14:43] Evaluate_01: epoch = 0500 train time = 8 s train loss = 0.348136 train acc = 0.9000, test acc = 0.2753


100%|██████████| 501/501 [00:08<00:00, 57.76it/s]


...

[2023-05-30 02:17:00] Evaluate_01: epoch = 0500 train time = 8 s train loss = 0.000427 train acc = 1.0000, test acc = 0.3248


100%|██████████| 501/501 [00:08<00:00, 56.47it/s]


[2023-05-30 02:17:09] Evaluate_01: epoch = 0500 train time = 8 s train loss = 0.000306 train acc = 1.0000, test acc = 0.3237


100%|██████████| 501/501 [00:09<00:00, 54.87it/s]

[2023-05-30 02:17:18] Evaluate_01: epoch = 0500 train time = 9 s train loss = 0.000072 train acc = 1.0000, test acc = 0.3262


In [13]:
def objective(trial):
    params = {
              'learning_rate': trial.suggest_float('learning_rate', 8e-4, 4e-2),
              }

    torch.manual_seed(0)
    model = AlexNet(3,10).to('cuda')
    args = argparse.Namespace(lr_net=str(params['learning_rate']), device='cuda', epoch_eval_train=str(3000),batch_train=100,dataset='cifar10',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True))                        
    model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)

    return acc_test

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20)

[I 2023-05-30 01:58:50,793] A new study created in memory with name: no-name-37b6b398-9c9f-458e-b060-89c4733957d3
100%|██████████| 3001/3001 [00:42<00:00, 70.97it/s]
[I 2023-05-30 01:59:33,096] Trial 0 finished with value: 0.3284 and parameters: {'learning_rate': 0.02310179288511339}. Best is trial 0 with value: 0.3284.


[2023-05-30 01:59:33] Evaluate_01: epoch = 3000 train time = 42 s train loss = 0.000437 train acc = 1.0000, test acc = 0.3284


100%|██████████| 3001/3001 [00:42<00:00, 70.59it/s]
[I 2023-05-30 02:00:15,627] Trial 1 finished with value: 0.3269 and parameters: {'learning_rate': 0.018072629718177063}. Best is trial 0 with value: 0.3284.


[2023-05-30 02:00:15] Evaluate_01: epoch = 3000 train time = 42 s train loss = 0.000784 train acc = 1.0000, test acc = 0.3269


100%|██████████| 3001/3001 [00:42<00:00, 70.23it/s]
[I 2023-05-30 02:00:58,372] Trial 2 finished with value: 0.1 and parameters: {'learning_rate': 0.0372786551854656}. Best is trial 0 with value: 0.3284.


...

[2023-05-30 02:11:03] Evaluate_01: epoch = 3000 train time = 43 s train loss = 0.001276 train acc = 1.0000, test acc = 0.3374


100%|██████████| 3001/3001 [00:42<00:00, 71.43it/s]
[I 2023-05-30 02:11:45,433] Trial 17 finished with value: 0.2966 and parameters: {'learning_rate': 0.009124869761263386}. Best is trial 15 with value: 0.3561.


[2023-05-30 02:11:45] Evaluate_01: epoch = 3000 train time = 42 s train loss = 0.005263 train acc = 1.0000, test acc = 0.2966


 70%|██████▉   | 2092/3001 [00:30<00:13, 68.65it/s]
[W 2023-05-30 02:12:15,922] Trial 18 failed with parameters: {'learning_rate': 0.006584268801467397} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_32/4138845070.py", line 9, in objective
    model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)
  File "/notebooks/mtt_distillation/utils.py", line 371, in evaluate_synset
    loss_train, acc_train = epoch('train', trainloader, net, optimizer, criterion, args, aug=True, texture=texture)
  File "/notebooks/mtt_distillation/utils.py", line 334, in epoch
    acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
KeyboardInterrupt
[W 2023-05-30 02:12:15,925] Trial 18 failed with value None.


KeyboardInterrupt: 

In [9]:
model = AlexNet(3,100).to('cuda')
args = argparse.Namespace(lr_net='.01', device='cuda', epoch_eval_train=str(1300),batch_train=512,dataset='cifar100',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)) #, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)

100%|██████████| 1301/1301 [06:44<00:00,  3.22it/s]

[2023-05-26 16:00:45] Evaluate_01: epoch = 1300 train time = 404 s train loss = nan train acc = 0.0100, test acc = 0.0100


In [ ]:
for i in range(3,5):
        torch.manual_seed(i)
        model = AlexNet(3,10)
        DistilledPruning(model, 'dist_c10_ipc10_an_seed' + str(i), path, images_train, labels_train, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs_distilled = 3000, num_epochs_real = 60, k = 0, amount = .2, save_model = False, validate = True, seed = 0, reinit = False, reinit_model = None, distilled_lr = .007)

Distilled Pruning Iteration  0


  0%|          | 0/3001 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 3001/3001 [00:49<00:00, 60.10it/s]


[2023-06-01 07:10:31] Evaluate_01: epoch = 3000 train time = 49 s train loss = 0.000194 train acc = 1.0000, test acc = 0.3179


/tmp/ipykernel_32/1718027906.py:259: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save(path + name + '_log', np.array([accs, zeros, totals, reinit_acc]))


Test Accuracy: 0.8141
Number of Zero Weights: 374263.0
Total Number of Weights: 1871232.0
Sparsity 0.2000088711608181
Distilled Pruning Iteration  1


...

Test Accuracy: 0.5869
Number of Zero Weights: 1864162.0
Total Number of Weights: 1871232.0
Sparsity 0.9962217405431288
Distilled Pruning Iteration  25


100%|██████████| 3001/3001 [00:46<00:00, 64.32it/s]


[2023-06-01 14:40:54] Evaluate_26: epoch = 3000 train time = 46 s train loss = 2.302458 train acc = 0.1400, test acc = 0.1659
Test Accuracy: 0.1
Number of Zero Weights: 1867612.0
Total Number of Weights: 1871232.0
Sparsity 0.9980654456529174
Distilled Pruning Iteration  28


100%|██████████| 3001/3001 [00:31<00:00, 95.70it/s] 


[2023-06-01 15:04:22] Evaluate_29: epoch = 3000 train time = 31 s train loss = 2.302585 train acc = 0.1000, test acc = 0.1000


In [9]:
for i in range(1,2):
    torch.manual_seed(i)
    model = get_network('ConvNetW128', 3,10)
    LotteryTicketRewinding(model, 'ltr_c10_seed' + str(i), path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, k = 1, amount = .2, save_model = False, seed = i, reinit = False)

LTR Iteration: 1
Number of Zero Weights: 63770.0
Total Number of Weights: 318848.0
Sparsity 0.20000125451625853
Test Accuracy: 0.8143
LTR Iteration: 2
Number of Zero Weights: 114786.0
Total Number of Weights: 318848.0
Sparsity 0.36000225812926534
Test Accuracy: 0.8132
LTR Iteration: 3
Number of Zero Weights: 155598.0
Total Number of Weights: 318848.0
Sparsity 0.48800055198715375
Test Accuracy: 0.8118
LTR Iteration: 4
Number of Zero Weights: 188248.0
Total Number of Weights: 318848.0
Sparsity 0.590400441589723
Test Accuracy: 0.8094
LTR Iteration: 5
Number of Zero Weights: 214368.0
Total Number of Weights: 318848.0
Sparsity 0.6723203532717784
Test Accuracy: 0.8101
LTR Iteration: 6
Number of Zero Weights: 235264.0
Total Number of Weights: 318848.0
Sparsity 0.7378562826174228
Test Accuracy: 0.806
LTR Iteration: 7
Number of Zero Weights: 251981.0
Total Number of Weights: 318848.0
Sparsity 0.7902856533520675
Test Accuracy: 0.8042
LTR Iteration: 8
Number of Zero Weights: 265354.0
Total Number

In [6]:
for i in range(1):
    torch.manual_seed(i)
    model = get_network('ConvNetW128', 3,100)
    RandomPruning(model, 'random_c10_seed' + str(i), path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, amount = .2, save_model = False, seed = i)

Random Pruning Iteration: 1
Number of Zero Weights: 100634.0
Total Number of Weights: 503168.0
Sparsity 0.2000007949631137
Test Accuracy: 0.8126
Random Pruning Iteration: 2
Number of Zero Weights: 181141.0
Total Number of Weights: 503168.0
Sparsity 0.3600010334520478
Test Accuracy: 0.7985
Random Pruning Iteration: 3
Number of Zero Weights: 245547.0
Total Number of Weights: 503168.0
Sparsity 0.4880020192063088
Test Accuracy: 0.7913
Random Pruning Iteration: 4
Number of Zero Weights: 297070.0
Total Number of Weights: 503168.0
Sparsity 0.590399230475706
Test Accuracy: 0.781
Random Pruning Iteration: 5
Number of Zero Weights: 338290.0
Total Number of Weights: 503168.0
Sparsity 0.6723201793436785
Test Accuracy: 0.7751
Random Pruning Iteration: 6
Number of Zero Weights: 371266.0
Total Number of Weights: 503168.0
Sparsity 0.7378569384380564
Test Accuracy: 0.7669
Random Pruning Iteration: 7
Number of Zero Weights: 397646.0
Total Number of Weights: 503168.0
Sparsity 0.7902847557873315
Test Accu

In [6]:
class RandomCIFAR10Dataset(Dataset):
    def __init__(self, root, transform=None, target_transform=None, num_samples=1000):
        self.cifar = torchvision.datasets.CIFAR10(root, train=True, download=True, transform=transform, target_transform=target_transform)
        self.num_samples = num_samples
        self.indices = torch.randperm(len(self.cifar))[:self.num_samples]

    def __getitem__(self, index):
        return self.cifar[self.indices[index]]

    def __len__(self):
        return self.num_samples
    
class EasyHardCIFAR10Dataset(Dataset):
    def __init__(self, root, pretrained_model, dataloader, transform=None, target_transform=None, mode='easy', num_samples=1000):
        assert mode in ['easy', 'hard'], "Mode should be 'easy' or 'hard'"
        transform = transforms.Compose([transforms.ToTensor()]) if transform is None else transform
        self.cifar = torchvision.datasets.CIFAR10(root, train=True, download=True, transform=transform, target_transform=target_transform)
        self.pretrained_model = pretrained_model
        self.mode = mode
        self.num_samples = num_samples
        self.dataloader = dataloader

        self.pretrained_model.eval()
        self.indices = []

        with torch.no_grad():
            for data, targets in self.dataloader:
                data = data.to('cuda')
                targets = targets.to('cuda')
                outputs = self.pretrained_model(data)
                _, preds = torch.max(outputs, 1)
                probs = torch.nn.functional.softmax(outputs, dim=1)

                # Get confidence scores and sort them
                confidence_scores, indices = torch.sort(probs.max(dim=1).values, descending=True)
                sorted_preds = preds[indices]
                sorted_targets = targets[indices]

                if mode == 'easy':
                    mask = sorted_preds == sorted_targets
                else:  # mode == 'hard'
                    mask = sorted_preds != sorted_targets

                # Select samples and add to the indices list
                self.indices.extend(indices[mask][:self.num_samples].tolist())

    def __getitem__(self, index):
        return self.cifar[self.indices[index]]

    def __len__(self):
        return len(self.indices)


easy_train_dataset_1000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, dataloader = train_loader, transform=None, target_transform=None, mode='easy', num_samples=1000)

easy_train_loader_1000 = torch.utils.data.DataLoader(dataset = easy_train_dataset_1000,
                                                    batch_size = batch_size,
                                                    shuffle = True)

hard_train_dataset_1000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, dataloader = train_loader, transform=None, target_transform=None, mode='hard', num_samples=1000)

hard_train_loader_1000 = torch.utils.data.DataLoader(dataset = hard_train_dataset_1000,
                                                    batch_size = batch_size,
                                                    shuffle = True)

random_train_dataset_1000 = RandomCIFAR10Dataset(root = './data', transform=None, target_transform=None, num_samples=1000)

random_train_loader_1000 = torch.utils.data.DataLoader(dataset = random_train_dataset_1000,
                                                    batch_size = batch_size,
                                                    shuffle = True)
"""

easy_train_dataset_5000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='easy', num_samples=5000)

easy_train_loader_5000 = torch.utils.data.DataLoader(dataset = easy_train_dataset_5000,
                                                    batch_size = batch_size,
                                                    shuffle = True)

hard_train_dataset_5000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='hard', num_samples=5000)

hard_train_loader_5000 = torch.utils.data.DataLoader(dataset = hard_train_dataset_5000,
                                                    batch_size = batch_size,
                                                    shuffle = True)

random_train_dataset_5000 = RandomCIFAR10Dataset(root = './data', transform=None, target_transform=None, num_samples=5000)

random_train_loader_5000 = torch.utils.data.DataLoader(dataset = random_train_dataset_5000,
                                                    batch_size = batch_size,
                                                    shuffle = True)
                                                    
easy_train_dataset_25000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='easy', num_samples=25000)

easy_train_loader_25000 = torch.utils.data.DataLoader(dataset = easy_train_dataset_25000,
                                                    batch_size = batch_size,
                                                    shuffle = True)

hard_train_dataset_25000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='hard', num_samples=25000)

hard_train_loader_25000 = torch.utils.data.DataLoader(dataset = hard_train_dataset_25000,
                                                    batch_size = batch_size,
                                                    shuffle = True)

random_train_dataset_25000 = RandomCIFAR10Dataset(root = './data', transform=None, target_transform=None, num_samples=25000)

random_train_loader_25000 = torch.utils.data.DataLoader(dataset = random_train_dataset_25000,
                                                    batch_size = batch_size,
                                                    shuffle = True)
"""

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


"\n\neasy_train_dataset_5000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='easy', num_samples=5000)\n\neasy_train_loader_5000 = torch.utils.data.DataLoader(dataset = easy_train_dataset_5000,\n                                                    batch_size = batch_size,\n                                                    shuffle = True)\n\nhard_train_dataset_5000 = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='hard', num_samples=5000)\n\nhard_train_loader_5000 = torch.utils.data.DataLoader(dataset = hard_train_dataset_5000,\n                                                    batch_size = batch_size,\n                                                    shuffle = True)\n\nrandom_train_dataset_5000 = RandomCIFAR10Dataset(root = './data', transform=None, target_transform=None, num_samples=5000)\n\nrandom_train_loader_5000 = torch.utils.da

In [11]:
def train(model,train_loader, num_epochs, lr = .0008, weight_decay = .0008, gamma = .15, milestones = [50,65,80]):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = weight_decay)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    cost = nn.CrossEntropyLoss()
    scheduler = MultiStepLR(optimizer, milestones=milestones, gamma= gamma)
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):  
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            loss = cost(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
            
def test(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Test the model
    model.eval()
    model.to(device)
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader): 
            images, labels = images.to(device), labels.to(device)
            test_output = model(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            correct += (pred_y == labels).sum().item()
            total += labels.size(0)
        accuracy = correct / total

    print('Test Accuracy:', accuracy)
    return accuracy

def get_parameters_to_prune(model):
    parameters_to_prune = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))
    return tuple(parameters_to_prune)

def sparsity_print(model):
    prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.L1Unstructured,amount=0)
    zero = total = 0
    for module, _ in get_parameters_to_prune(model):
        zero += float(torch.sum(module.weight == 0))
        total += float(module.weight.nelement())
    print('Number of Zero Weights:', zero)
    print('Total Number of Weights:', total)
    print('Sparsity', zero/total)
    #TODO: Implement Node Sparsity
    return zero, total

#Standard IMP with Weight Rewinding, name is a string that allows us to save models/logs appropriately, path is the location of folder we save to, start_iter should be 0 but if a experiment stops halfway through it allows us to begin there,
#amount = % params pruned each pruning iteration, save_model downloads each model at every iter, reinit is boolean value if we want to test results on reinitialized weights
#reinit_model is the specific model that holds the reinitialized weights
def LotteryTicketRewinding(model, name, path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, k = 1, amount = .2, save_model = True, seed = 0, reinit = False, reinit_model = None):
    torch.manual_seed(seed)
    zeros = []
    totals = []
    acc = []
    reinit_acc = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
    #Create Rewind Weights after training K epochs
    train(model, train_loader,num_epochs = k)
    torch.save(model.state_dict(), path + name + '_RewindWeights' + '_' + str(k))
    model_rewind = copy.deepcopy(model).to(device)
    
    #Finish off the pretraining
    train(model, train_loader,num_epochs = num_epochs - k)

    #Lottery Ticket Rewinding: Prune, Rewind, Train
    for i in range(start_iter,end_iter):
        print('LTR Iteration:', i+1)
        #Prune
        prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.L1Unstructured,amount=amount)
        #Rewind Weights
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
        #Train Weights
        train(model, train_loader,num_epochs = num_epochs)
        
        #Log Results
        zero, total = sparsity_print(model)
        zeros.append(zero)
        totals.append(total)
        acc.append(test(model, test_loader))
        if save_model:
            torch.save(model.state_dict(), path + name + '_iter' + str(i+1))
            
        if reinit:
            #Rewind Weights
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_reinit = get_parameters_to_prune(reinit_model)[idx][0]
                    module.weight_orig.copy_(module_reinit.weight)
                    
            train(model, train_loader,num_epochs = num_epochs)
            reinit_acc.append(test(model, test_loader))
            
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                    module.weight_orig.copy_(module_rewind.weight)
        else:
            reinit_acc.append(0)
            
        np.save(path + name + '_log', np.array([acc,zeros,totals,reinit_acc]))
    
    pass
  
def DataDiet(model, name, path, subset_loader, train_loader, test_loader, start_iter = 0, end_iter = 30, subset_epochs = 120, num_epochs = 60, k = 1, amount = .2, save_model = True, seed = 0, reinit = False, reinit_model = None):
    torch.manual_seed(seed)
    zeros = []
    totals = []
    acc = []
    reinit_acc = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
    #Create Rewind Weights after training K epochs
    train(model, train_loader,num_epochs = k)
    torch.save(model.state_dict(), path + name + '_RewindWeights' + '_' + str(k))
    model_rewind = copy.deepcopy(model).to(device)
    
    #Finish off the pretraining
    train(model, train_loader,num_epochs = num_epochs - k)

    #Lottery Ticket Rewinding: Prune, Rewind, Train
    for i in range(start_iter,end_iter):
        print('LTR Iteration:', i+1)
        #Rewind Weights
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
        #Train Weights
        train(model, subset_loader,num_epochs = subset_epochs)
        prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.L1Unstructured,amount=amount)
        
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
                
        train(model, train_loader,num_epochs = num_epochs)
        #Log Results
        zero, total = sparsity_print(model)
        zeros.append(zero)
        totals.append(total)
        acc.append(test(model, test_loader))
        if save_model:
            torch.save(model.state_dict(), path + name + '_iter' + str(i+1))
            
        if reinit:
            #Rewind Weights
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_reinit = get_parameters_to_prune(reinit_model)[idx][0]
                    module.weight_orig.copy_(module_reinit.weight)
                    
            train(model, train_loader,num_epochs = num_epochs)
            reinit_acc.append(test(model, test_loader))
            
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                    module.weight_orig.copy_(module_rewind.weight)
        else:
            reinit_acc.append(0)
            
        np.save(path + name + '_log', np.array([acc,zeros,totals,reinit_acc]))
    
    pass
#Generate full sparsity curve with retraining for random pruning, this is not a method to compute a single, high-sparisty model with random pruning. This generates and trains models at all sparsities for comparison
def RandomPruning(model, name, path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, amount = .2, save_model = True, seed = 0):
    torch.manual_seed(seed)
    zeros = []
    totals = []
    acc = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
    model_rewind = copy.deepcopy(model).to(device)
    
    for i in range(start_iter,end_iter):
        print('Random Pruning Iteration:', i+1)
        #Prune
        prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.RandomUnstructured,amount=amount)
        #Train Weights
        train(model, train_loader,num_epochs = num_epochs)
        
        #Log Results
        zero, total = sparsity_print(model)
        zeros.append(zero)
        totals.append(total)
        acc.append(test(model, test_loader))

            
        #Rewind Weights to save them
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
                
        if save_model:
            torch.save(model.state_dict(), path + name + '_iter' + str(i+1))
            
        np.save(path + name + '_log', np.array([acc,zeros,totals]))
    
#Generate full sparsity curve to compare with LTR
def DistilledPruning(model, name, path, images_train, labels_train, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs_distilled = 1000, num_epochs_real = 60, k = 0, amount = .2, save_model = True, validate = False, seed = 0, reinit = False, reinit_model = None, distilled_lr = .01):
    torch.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    accs = []
    zeros = []
    totals = []
    reinit_acc = []
    
    model_rewind = copy.deepcopy(model).to(device)
    torch.save(model.state_dict(), path + name + '_RewindWeights' + '_' + str(k))
    
    if k != 0:
        args = argparse.Namespace(lr_net=str(distilled_lr), device='cuda', epoch_eval_train=str(k),batch_train=512,dataset='cifar10',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)) #, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
        model_rewind, acc_train_list, acc_test = evaluate_synset(0, model_rewind,images_train,labels_train,test_loader,args)
        
    
    for i in range(start_iter,end_iter):
        print('Distilled Pruning Iteration ', i)
        args = argparse.Namespace(lr_net='.01', device='cuda', epoch_eval_train=str(num_epochs_distilled),batch_train=512,dataset='cifar10',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)) #, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
        #MTT Training on Distilled Data
        model, acc_train_list, acc_test = evaluate_synset(i+1, model,images_train,labels_train,test_loader,args)
        prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.L1Unstructured,amount=amount)
        #Rewind Weights
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
    
        if save_model:
            torch.save(model.state_dict(), path + name + '_iter' + str(i+1))
            
        if validate:
            train(model, train_loader,num_epochs = num_epochs_real)
            accs.append(test(model, test_loader))
            zero, total = sparsity_print(model)
            zeros.append(zero)
            totals.append(total)
            #Rewind Weights
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                    module.weight_orig.copy_(module_rewind.weight)
                    
            np.save(path + name + '_log', np.array([accs, zeros, totals, reinit_acc]))
        
        if reinit:
            #Rewind Weights to Reinit Model
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_reinit = get_parameters_to_prune(reinit_model)[idx][0]
                    module.weight_orig.copy_(module_reinit.weight)
                    
            train(model, train_loader,num_epochs = num_epochs_real)
            reinit_acc.append(test(model, test_loader))
            
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                    module.weight_orig.copy_(module_rewind.weight)
            np.save(path + name + '_log', np.array([accs, zeros, totals, reinit_acc]))
        else:
            reinit_acc.append(0)
            
    if not validate:
        train(model, train_loader,num_epochs = num_epochs_real)
        acc = (test(model, test_loader))
        zero, total = sparsity_print(model)
        np.save(path + name + '_log', np.array([acc, zero, total, reinit]))
    
path = './model_results_cifar10/'

In [12]:
for i in range(1):
    torch.manual_seed(i)
    model = AlexNet(3,10).to('cuda')
    DataDiet(model, 'diet_c10_easy1000_seed' + str(i), path, easy_train_loader_1000, train_loader, test_loader, start_iter = 0, end_iter = 30, subset_epochs = 120, num_epochs = 60, k = 1, amount = .2, save_model = False, seed = i, reinit = False)

KeyboardInterrupt: 

In [9]:
def objective(trial):
    params = {
              'learning_rate': trial.suggest_float('learning_rate', 8e-5, 4e-3),
              }

    torch.manual_seed(0)
    model = AlexNet(3,10).to('cuda')
    train(model, easy_train_loader_1000, 30, lr = params['learning_rate'])
    return test(model, test_loader)

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20)

model = AlexNet(3,10).to('cuda')
for i in range(10):
    train(model, easy_train_loader_1000, 30)
    test(model, test_loader)

[I 2023-05-31 18:00:22,872] A new study created in memory with name: no-name-122dc726-00e2-468f-abca-f4d2e31c164b
[I 2023-05-31 18:01:55,055] Trial 0 finished with value: 0.0768 and parameters: {'learning_rate': 0.0024562846250703882}. Best is trial 0 with value: 0.0768.


Test Accuracy: 0.0768


[I 2023-05-31 18:03:26,846] Trial 1 finished with value: 0.1542 and parameters: {'learning_rate': 0.0007407817399856925}. Best is trial 1 with value: 0.1542.


Test Accuracy: 0.1542


[I 2023-05-31 18:04:58,492] Trial 2 finished with value: 0.0948 and parameters: {'learning_rate': 0.003599480692390955}. Best is trial 1 with value: 0.1542.


...

Test Accuracy: 0.1123


[I 2023-05-31 18:09:33,040] Trial 5 finished with value: 0.1187 and parameters: {'learning_rate': 0.003938279254994967}. Best is trial 1 with value: 0.1542.


Test Accuracy: 0.1187


[W 2023-05-31 18:10:18,718] Trial 6 failed with parameters: {'learning_rate': 0.0032565711060557915} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_32/1021480039.py", line 8, in objective
    train(model, easy_train_loader_1000, 30, lr = params['learning_rate'])
  File "/tmp/ipykernel_32/96052406.py", line 8, in train
    for i, (images, labels) in enumerate(train_loader):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 681, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 721, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.da

KeyboardInterrupt: 